# Demo

Connecting to the knowledge base

In [1]:
from coupled_modelling import KnowledgeBase

kb = KnowledgeBase('http://127.0.0.1:5000/api/v1.0/')

kb

Importing a couple of existing coupled systems from JSON files

In [2]:
import os
import json

main_dir = os.path.dirname(os.path.abspath(''))

input_path_1 = os.path.join(
    main_dir,
    'mixed_fid_models/',
    'ProjectParametersCoSimFSI.json')

input_path_2 = os.path.join(
    main_dir,
    'FSI_Mok/',
    'ProjectParametersCoSim.json')

inputs = {
    'mixed_fid_models': input_path_1,
    'FSI_Mok': input_path_2
}

for label, path in inputs.items():
    with open(path) as f:
        data = json.load(f)
    kb.import_kratos(label, data)

The knowledge base creates a high level class for each nested property of the coupled system and infers a specific subclass of the high level class if an instance in the coupled system has a unique set of value types for its properties. Let's look at the class hierarchy in the knowledge base

In [3]:
kb.get_class_hierarchy()

{'communication_format': [],
 'connect_to': [],
 'convergence_accelerators': ['convergence_accelerators_1',
  'convergence_accelerators_2'],
 'convergence_criteria': ['convergence_criteria_1', 'convergence_criteria_2'],
 'coupled_system': ['coupled_system_1', 'coupled_system_2'],
 'coupling_sequence': ['coupling_sequence_1',
  'coupling_sequence_2',
  'coupling_sequence_3'],
 'data': ['data_1', 'data_2'],
 'data_name': [],
 'data_transfer_operator': [],
 'data_transfer_operator_options': [],
 'data_transfer_operators': ['data_transfer_operators_1',
  'data_transfer_operators_2'],
 'export_data': [],
 'from_solver': [],
 'from_solver_data': [],
 'import_data': [],
 'import_meshes': [],
 'input_data_list': ['input_data_list_1', 'input_data_list_2'],
 'input_file': [],
 'io_settings': ['io_settings_1'],
 'location': [],
 'mapper_settings': ['mapper_settings_1', 'mapper_settings_2'],
 'mapper_type': [],
 'model_part_name': [],
 'name': [],
 'output_data_list': ['output_data_list_1', 'outpu

Look into properties of the *coupled_system_1* subclass specifically. Each item in the list has a name of the property, its cardinality, i.e. the number of values, and its value type

In [4]:
coupled_system_class_1 = kb.get_class('coupled_system_1')
coupled_system_class_1.get_properties()

[{'cardinality': 1,
  'property': 'has_problem_data',
  'value': ['problem_data_1']},
 {'cardinality': 1,
  'property': 'has_solver_settings',
  'value': ['solver_settings_1']}]

Let's compare it with the second subclass

In [5]:
coupled_system_class_2 = kb.get_class('coupled_system_2')
coupled_system_class_2.get_properties()

[{'cardinality': 1,
  'property': 'has_problem_data',
  'value': ['problem_data_1']},
 {'cardinality': 1,
  'property': 'has_solver_settings',
  'value': ['solver_settings_2']}]

We see that both classes have similar problem data but different solver settings. Let's check the difference in the solver settings

In [6]:
solver_settings_class_1 = kb.get_class('solver_settings_1')
solver_settings_class_1.get_properties()

[{'cardinality': 1, 'property': 'has_echo_level', 'value': 'int'},
 {'cardinality': 1, 'property': 'has_solvers', 'value': ['solvers_1']},
 {'cardinality': 1, 'property': 'has_solvers', 'value': ['solvers_2']},
 {'cardinality': 1, 'property': 'has_type', 'value': ['type']},
 {'cardinality': 1,
  'property': 'has_convergence_criteria',
  'value': ['convergence_criteria_1']},
 {'cardinality': 1,
  'property': 'has_data_transfer_operators',
  'value': ['data_transfer_operators_1']},
 {'cardinality': 1, 'property': 'has_num_coupling_iterations', 'value': 'int'},
 {'cardinality': 1,
  'property': 'has_coupling_sequence',
  'value': ['coupling_sequence_1']},
 {'cardinality': 1,
  'property': 'has_coupling_sequence',
  'value': ['coupling_sequence_2']},
 {'cardinality': 1,
  'property': 'has_convergence_accelerators',
  'value': ['convergence_accelerators_1']}]

In [7]:
solver_settings_class_2 = kb.get_class('solver_settings_2')
solver_settings_class_2.get_properties()

[{'cardinality': 1, 'property': 'has_echo_level', 'value': 'int'},
 {'cardinality': 1, 'property': 'has_solvers', 'value': ['solvers_3']},
 {'cardinality': 1, 'property': 'has_solvers', 'value': ['solvers_4']},
 {'cardinality': 1, 'property': 'has_type', 'value': ['type']},
 {'cardinality': 1,
  'property': 'has_convergence_criteria',
  'value': ['convergence_criteria_2']},
 {'cardinality': 1,
  'property': 'has_data_transfer_operators',
  'value': ['data_transfer_operators_2']},
 {'cardinality': 1, 'property': 'has_num_coupling_iterations', 'value': 'int'},
 {'cardinality': 1,
  'property': 'has_coupling_sequence',
  'value': ['coupling_sequence_1']},
 {'cardinality': 1,
  'property': 'has_coupling_sequence',
  'value': ['coupling_sequence_3']},
 {'cardinality': 1,
  'property': 'has_convergence_accelerators',
  'value': ['convergence_accelerators_2']}]

We see that they have different solvers, convergence criteria, data transfer operators, second items in coupling sequences and convergence accelerators.

To avoid subsequent quering nested class properties, the *.get_properties()* method has an optional *depth* parameter, which specifies how many levels of nested classes the method should retrieve properties for.

In [8]:
coupled_system_class_1.get_properties(2)

[{'cardinality': 1,
  'property': 'has_problem_data',
  'value': [{'problem_data_1': [{'cardinality': 1,
      'property': 'has_echo_level',
      'value': 'int'},
     {'cardinality': 1, 'property': 'has_end_time', 'value': 'float'},
     {'cardinality': 1, 'property': 'has_start_time', 'value': 'float'},
     {'cardinality': 1, 'property': 'has_print_colors', 'value': 'bool'},
     {'cardinality': 1,
      'property': 'has_parallel_type',
      'value': ['parallel_type']}]}]},
 {'cardinality': 1,
  'property': 'has_solver_settings',
  'value': [{'solver_settings_1': [{'cardinality': 1,
      'property': 'has_echo_level',
      'value': 'int'},
     {'cardinality': 1, 'property': 'has_solvers', 'value': ['solvers_1']},
     {'cardinality': 1, 'property': 'has_solvers', 'value': ['solvers_2']},
     {'cardinality': 1, 'property': 'has_type', 'value': ['type']},
     {'cardinality': 1,
      'property': 'has_convergence_criteria',
      'value': ['convergence_criteria_1']},
     {'cardi

We can check instances of this class

In [9]:
coupled_system_class_1.get_instances()

['instance_1', 'instance_82', 'instance_123']

Checking properties of the first instance, which is called *mixed_fid_models*

In [ ]:
mixed_fid_models = kb.get_instance(coupled_system_class_1.instances[0])
mixed_fid_models.get_properties()

{'label': 'mixed_fid_models',
 'problem_data': 'instance_2',
 'solver_settings': 'instance_4'}

Instance properties can be retrieved recursively too

In [ ]:
mixed_fid_models.get_properties(2)

{'label': 'mixed_fid_models',
 'problem_data': {'instance_2': {'echo_level': 0,
   'end_time': 1.0,
   'parallel_type': 'OpenMP',
   'print_colors': True,
   'start_time': 0.0}},
 'solver_settings': {'instance_4': {'convergence_accelerators': ['instance_10'],
   'convergence_criteria': ['instance_14'],
   'coupling_sequence': ['instance_16', 'instance_17'],
   'data_transfer_operators': 'instance_6',
   'echo_level': 3,
   'num_coupling_iterations': 20,
   'solvers': ['instance_23', 'instance_36'],
   'type': 'coupled_solvers.gauss_seidel_strong'}}}

Checking instances of the second class

In [12]:
coupled_system_class_2.get_instances()

['instance_43', 'instance_103', 'instance_144']

In [ ]:
FSI_Mok = kb.get_instance(coupled_system_class_2.instances[0])
FSI_Mok.get_properties(2)

{'label': 'FSI_Mok',
 'problem_data': {'instance_44': {'echo_level': 0,
   'end_time': 25.0,
   'parallel_type': 'OpenMP',
   'print_colors': True,
   'start_time': 0.0}},
 'solver_settings': {'instance_45': {'convergence_accelerators': ['instance_46'],
   'convergence_criteria': ['instance_50'],
   'coupling_sequence': ['instance_53', 'instance_54'],
   'data_transfer_operators': 'instance_51',
   'echo_level': 1,
   'num_coupling_iterations': 12,
   'solvers': ['instance_62', 'instance_73'],
   'type': 'coupled_solvers.gauss_seidel_strong'}}}

Creating a new coupled system

In [15]:
Onera_FSI = kb.create_coupled('Onera_FSI')

Onera_FSI

We guess that the new couple system is similar to *mixed_fid_models* and can reuse its properties. All values of the problem data from *mixed_fid_models* are the same, except of the *echo_level* value. Thus, we copy problem data from the existing coupled system with the new coupled system as a parent instance and setting a new *echo_level* value to the new problem data

In [16]:
problem_data_old = next(iter(mixed_fid_models.properties['problem_data']))
problem_data_old = kb.get_instance(problem_data_old)

problem_data_new = problem_data_old.make_copy(
    Onera_FSI,
    {'echo_level': 2})

problem_data_new

The copying method does not copy nested instances. Thus, we can copy the solver settings from the existing coupled system with the old *num_coupling_iterations* and *type*, but with a new *echo_level*. Afterwards, we will create instances of convergence accelerators, convergence criteria, coupling sequence, data transfer_operators and solvers, and add them to the mew solver settings

In [17]:
solver_settings_old = next(iter(mixed_fid_models.properties['solver_settings']))
solver_settings_old = kb.get_instance(solver_settings_old)

solver_settings_new = solver_settings_old.make_copy(
    Onera_FSI,
    {'echo_level': 4})

solver_settings_new

Checking existing solver settings

In [18]:
solver_settings_old.get_properties(2)

{'convergence_accelerators': [{'instance_10': {'data_name': 'pitch_angle',
    'solver': 'low_fid_fluid',
    'type': 'aitken'}}],
 'convergence_criteria': [{'instance_14': {'abs_tolerance': 1e-07,
    'data_name': 'pitch_angle',
    'rel_tolerance': 1e-05,
    'solver': 'low_fid_fluid',
    'type': 'relative_norm_previous_residual'}}],
 'coupling_sequence': [{'instance_16': {'name': 'low_fid_fluid'}},
  {'instance_17': {'input_data_list': ['instance_19'],
    'name': 'spring_structure',
    'output_data_list': ['instance_22']}}],
 'data_transfer_operators': {'instance_6': {'label': 'direct_transfer',
   'mapper_settings': 'instance_8',
   'type': 'kratos_mapping'}},
 'echo_level': 3,
 'num_coupling_iterations': 20,
 'solvers': [{'instance_23': {'data': ['instance_31', 'instance_34'],
    'io_settings': 'instance_27',
    'label': 'low_fid_fluid',
    'solver_wrapper_settings': 'instance_25',
    'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}},
  {'instance_36': {

Only *type* of the existing convergence accelerators can be reused, thus, we create new convergence accelerators for the new coupled system from scratch, passing the class, the parent instance and new property values

In [19]:
convergence_accelerators_old = next(iter(solver_settings_old.properties['convergence_accelerators'][0]))
convergence_accelerators_old = kb.get_instance(convergence_accelerators_old)

convergence_accelerators_new = kb.create_instance(
    'convergence_accelerators',
    solver_settings_new,
    {'data_name': 'displacements', 'solver': 'CFD', 'type': 'aitken'})

convergence_accelerators_new

Copying convergence criteria, passing new *data_name*, *solver* and *type*

In [20]:
convergence_criteria_old = next(iter(solver_settings_old.properties['convergence_criteria'][0]))
convergence_criteria_old = kb.get_instance(convergence_criteria_old)

convergence_criteria_new = convergence_criteria_old.make_copy(solver_settings_new, {
    'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'})

convergence_criteria_new

We create a new coupling sequence by creating its elements one by one

In [21]:
coupling_sequence_new_0 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'CFD'})

coupling_sequence_new_0

Creating another coupling sequence

In [22]:
coupling_sequence_new_1 = kb.create_instance('coupling_sequence', solver_settings_new, {
    'name': 'SM'})

coupling_sequence_new_1

We create new nested instances for the second item in the new coupling sequence by copying nested instances of the second item of the old coupling sequence. Getting the second item of the old coupling sequence first

In [23]:
coupling_sequence_old_1 = next(iter(solver_settings_old.properties['coupling_sequence'][1]))
coupling_sequence_old_1 = kb.get_instance(coupling_sequence_old_1)
coupling_sequence_old_1.get_properties(2)

{'input_data_list': [{'instance_19': {'data': 'lift_force',
    'data_transfer_operator': 'direct_transfer',
    'from_solver': 'low_fid_fluid',
    'from_solver_data': 'lift_force'}}],
 'name': 'spring_structure',
 'output_data_list': [{'instance_22': {'data': 'pitch_angle',
    'data_transfer_operator': 'direct_transfer',
    'to_solver': 'low_fid_fluid',
    'to_solver_data': 'pitch_angle'}}]}

In [24]:
input_data_list_old = next(iter(coupling_sequence_old_1.properties['input_data_list'][0]))
input_data_list_old = kb.get_instance(input_data_list_old)

input_data_list_new = input_data_list_old.make_copy(coupling_sequence_new_1, {
    'data_transfer_operator': 'mapping_operation',
    'from_solver': 'CFD',
    'data_transfer_operator_options': 'use_transpose'})

input_data_list_new

Creating a new output data list

In [25]:
output_data_list_old = next(iter(coupling_sequence_old_1.properties['output_data_list'][0]))
output_data_list_old = kb.get_instance(output_data_list_old)

output_data_list_new = kb.create_instance('output_data_list', coupling_sequence_new_1, {
    'data': 'displacements',
    'data_transfer_operator': 'mapping_operation',
    'to_solver': 'CFD',
    'to_solver_data': 'displacements'})

output_data_list_new

Creating a mapping operation instance as the data transfer operator

In [26]:
mapping_operation = kb.create_instance('data_transfer_operators', solver_settings_new, {
    'label': 'mapping_operation',
    'type': 'kratos_mapping'})

mapping_operation

Creating new mapper settings

In [27]:
mapper_settings = kb.create_instance('mapper_settings', mapping_operation, {
    'mapper_type': 'nearest_neighbor',
    'use_initial_configuration': True})

mapper_settings

Checking the first existing solver

In [28]:
solver_old_0 = next(iter(solver_settings_old.properties['solvers'][0]))
solver_old_0 = kb.get_instance(solver_old_0)
solver_old_0.get_properties(2)

{'data': [{'instance_31': {'label': 'pitch_angle',
    'location': 'node_historical',
    'model_part_name': 'single_node_mesh_f',
    'variable_name': 'SCALAR_DISPLACEMENT'}},
  {'instance_34': {'label': 'lift_force',
    'location': 'node_historical',
    'model_part_name': 'single_node_mesh_f',
    'variable_name': 'SCALAR_FORCE'}}],
 'io_settings': {'instance_27': {'communication_format': 'file',
   'connect_to': 'run_fluid',
   'echo_level': 4,
   'type': 'kratos_co_sim_io'}},
 'label': 'low_fid_fluid',
 'solver_wrapper_settings': {'instance_25': {'export_data': ['pitch_angle'],
   'import_data': ['lift_force'],
   'import_meshes': ['single_node_mesh_f']}},
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

In the new soupled system, we apply a CFD solver. Thus, we creat an instance of the CFD solver by copying the first solver

In [29]:
CFD = solver_old_0.make_copy(solver_settings_new, {
    'label': 'CFD'})

CFD

The old data doesn't fit to the new solver. Creating displacements data for the CFD solver instance

In [30]:
displacements_0 = kb.create_instance('data', CFD, {
    'label': 'displacements',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'MESH_DISPLACEMENT'})

displacements_0

Creating lift force data instance for the CFD solver

In [31]:
lift_force_0 = kb.create_instance('data', CFD, {
    'label': 'lift_force',
    'dimensions': 3,
    'model_part_name': 'WING',
    'variable_name': 'REACTION'})

lift_force_0

Copying io settings with a new *connect_to* value

In [32]:
io_settings_old = next(iter(solver_old_0.properties['io_settings']))
io_settings_old = kb.get_instance(io_settings_old)

io_settings_new = io_settings_old.make_copy(CFD, {
    'connect_to': 'run_SU2'})

io_settings_new

Copying solver wrapper settings

In [33]:
solver_wrapper_settings_old_0 = next(iter(solver_old_0.properties['solver_wrapper_settings']))
solver_wrapper_settings_old_0 = kb.get_instance(solver_wrapper_settings_old_0)

solver_wrapper_settings_new_0 = solver_wrapper_settings_old_0.make_copy(CFD, {
    'export_data': 'displacements',
    'import_meshes': 'WING'})

solver_wrapper_settings_new_0

Checking another existing solver

In [34]:
solver_old_1 = next(iter(solver_settings_old.properties['solvers'][1]))
solver_old_1 = kb.get_instance(solver_old_1)
solver_old_1.get_properties(2)

{'data': [{'instance_41': {'label': 'pitch_angle',
    'location': 'node_historical',
    'model_part_name': 'single_node_mesh_s',
    'variable_name': 'SCALAR_DISPLACEMENT'}},
  {'instance_42': {'label': 'lift_force',
    'location': 'node_historical',
    'model_part_name': 'single_node_mesh_s',
    'variable_name': 'SCALAR_FORCE'}}],
 'io_settings': {'instance_39': {'communication_format': 'file',
   'connect_to': 'run_structure',
   'echo_level': 4,
   'type': 'kratos_co_sim_io'}},
 'label': 'spring_structure',
 'solver_wrapper_settings': {'instance_37': {'export_data': ['lift_force'],
   'import_data': ['pitch_angle'],
   'import_meshes': ['single_node_mesh_s']}},
 'type': 'solver_wrappers.external.remote_controlled_solver_wrapper'}

As the second solver for the new soupled system, we creat an instance of the SM solver

In [35]:
SM = kb.create_instance('solvers', solver_settings_new, {
    'label': 'SM',
    'type': 'solver_wrappers.kratos.structural_mechanics_wrapper'})

SM

Coping data for the second new solver from the first one

In [36]:
displacements_1 = displacements_0.make_copy(SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'DISPLACEMENT'})

displacements_1

Copying another data from the first solver

In [37]:
lift_force_1 = lift_force_0.make_copy(SM, {
    'model_part_name': 'Structure.interface',
    'variable_name': 'POINT_LOAD'})

lift_force_1

Creating solver wrapper settings for the second new solver

In [38]:
solver_wrapper_settings_new_1 = kb.create_instance('solver_wrapper_settings', SM, {
    'input_file': 'ProjectParametersSM'})

solver_wrapper_settings_new_1

Exporting created coupled system in a Kratos-compatible JSON format

In [39]:
export = Onera_FSI.export_kratos()

export

{'problem_data': {'echo_level': 2,
  'end_time': 1.0,
  'parallel_type': 'OpenMP',
  'print_colors': True,
  'start_time': 0.0},
 'solver_settings': {'convergence_accelerators': [{'data_name': 'displacements',
    'solver': 'CFD',
    'type': 'aitken'}],
  'convergence_criteria': [{'abs_tolerance': 1e-07,
    'data_name': 'displacements',
    'rel_tolerance': 1e-05,
    'solver': 'CFD',
    'type': 'relative_norm_initial_residual'}],
  'coupling_sequence': [{'name': 'CFD'},
   {'input_data_list': [{'data': 'lift_force',
      'data_transfer_operator': 'mapping_operation',
      'data_transfer_operator_options': ['use_transpose'],
      'from_solver': 'CFD',
      'from_solver_data': 'lift_force'}],
    'name': 'SM',
    'output_data_list': [{'data': 'displacements',
      'data_transfer_operator': 'mapping_operation',
      'to_solver': 'CFD',
      'to_solver_data': 'displacements'}]}],
  'data_transfer_operators': {'mapping_operation': {'mapper_settings': {'mapper_type': 'nearest_nei

Saving exported data into a file

In [40]:
export_path = os.path.join(
    os.path.abspath(''),
    'export_onera_fsi.json')

with open(export_path, 'w') as file:
    json.dump(export, file, indent=2)

Inferring new ontology classes from the created coupled system

In [41]:
Onera_FSI.infer_classes()

''

Updating the knowledge base

In [42]:
kb.save()

''

Getting a local copy of the knowledge base

In [43]:
onto_path = os.path.join(
    os.path.abspath(''),
    'demo_client.owl')

kb.save_locally(onto_path)